# 최근 한달 일일 검색량

In [26]:
import pandas as pd 
import collections
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import numpy as np

import math
from datetime import datetime 
from datetime import datetime, timedelta


from powernad.API.Campaign import *
from powernad.API.RelKwdStat import *
from powernad.API.AdGroup import *

import time

from time import sleep
from urllib.error import HTTPError

import urllib.request
from datetime import datetime, timedelta
import json

from tqdm.notebook import tqdm

In [27]:
#pip install jsonpickle

In [28]:
#pip install powernad --update

In [29]:
toda = datetime.now()
time_month = toda - relativedelta(months=1)
time_month = time_month.strftime('%Y-%m-%d')
time_month= str(time_month)


yesterday = toda - relativedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d')
yesterday = str(yesterday)



today = str(datetime.now().date())

In [30]:
dt_index = pd.date_range(start=time_month, end = yesterday)


date = pd.DataFrame(data=dt_index, columns=['날짜'])

In [31]:
# 네이버 디벨러퍼 API / 네이버 광고 API

In [32]:
BASE_URL = 'https://api.naver.com'
API_KEY = '*****'
SECRET_KEY = '*****'
CUSTOMER_ID = '****'

In [33]:
rel = RelKwdStat(BASE_URL, API_KEY, SECRET_KEY, CUSTOMER_ID)

def search_keyword(searchword):
    

    kwdDataList = rel.get_rel_kwd_stat_list(siteId=None, biztpId=None, hintKeywords=searchword, event=None, month=None, showDetail='1')


    kwd_result = (kwdDataList[0].relKeyword, #키워드
                 kwdDataList[0].monthlyPcQcCnt, #월간 검색수 (PC)
                 kwdDataList[0].monthlyMobileQcCnt, # 월간 검색수 (Mobile)
                 kwdDataList[0].monthlyPcQcCnt+kwdDataList[0].monthlyMobileQcCnt) # 월간 total 

    return(kwd_result[3])

client_id = "****" 
client_secret = "****"

dictionary_yester={}
dictionary_now={}

In [34]:
# lis = ['검색하고자 하는 키워드 입력']
lis = ['이대맛집']

In [35]:
error=[]
for i in tqdm(lis):  
    sleep(1)
    try:
        if type(search_keyword(i)) !=str : 
            searchword = i.replace(" ","")
            a = search_keyword(searchword)


            url = "https://openapi.naver.com/v1/datalab/search"
            body = "{\"startDate\":\""+time_month+"\",\"endDate\":\""+today+"\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\""+i+"\",\"keywords\":[\""+i+"\"]}]}";
            requested = urllib.request.Request(url)
            requested.add_header("X-Naver-Client-Id", client_id)
            requested.add_header("X-Naver-Client-Secret", client_secret)
            requested.add_header("Content-Type", "application/json")
            response = urllib.request.urlopen(requested, data=body.encode("utf-8"))
            rescode = response.getcode()

            if(rescode==200):
                response_body = response.read()
                output_data = response_body.decode('utf-8')
            else:
                print('Error code:'+ rescode)
                pass

            result = json.loads(output_data)


            date = [a['period'] for a in result['results'][0]['data']]


            aa = pd.DataFrame({'date':date, 
                          i:[a['ratio'] for a in result['results'][0]['data']],
                          })

            # 일일 데이터 계산 


            sleep(0.5)
            total = aa[i].sum()


            aa[i] = aa[i].apply(lambda x :((x / total)*float(a)))

        else : 
            pass

    except (TypeError, IndexError,KeyError,ValueError):
        print(" 타입 or 인덱스 에러,Value 에러:",i)
        error.append(i)
        pass


    except HTTPError:
        print('http 에러:', i )
        error.append(i)
        pass

        #여기부터 새로코딩함 
    try: 
        if type(search_keyword(i)) !=str : 
            searchword = i.replace(" ","")
            a = search_keyword(searchword)        

            dt_index = pd.date_range(start=time_month, end= yesterday)
            dt_list = dt_index.strftime("%Y-%m-%d").tolist()

            date = pd.DataFrame(data=dt_list, columns=['날짜'])

            spred = pd.merge(date,aa,left_on='날짜',right_on='date',how='outer')
            spred.drop(['date'],inplace=True, axis=1)

            spred.replace(np.nan,0,inplace=True)


            a=[] 

            for j in range(len(spred)) : 
                a.append(today)


            spred['수집날짜'] = a

            spred = spred[['날짜','수집날짜',i]]

            if i in dictionary_yester:

                pass

            else : 
                dictionary_yester[i]=[]



            dictionary_now[i]=spred

        else:
            pass

    except (TypeError, IndexError,KeyError,ValueError):

        pass

    sleep(0.5)
    try :
        if type(search_keyword(i)) !=str : 
            searchword = i.replace(" ","")
            a = search_keyword(searchword)        

            if dictionary_yester[i]==[]:
                dictionary_yester[i] = dictionary_now[i]
               # for k in range(3):
                #    sleep(1)
                 #   gc2.append_row(dictionary_now[i].iloc[-1-k,:].values.tolist()+[i])


            else : 
                pass 
        else:
            pass


    except (TypeError, IndexError,KeyError):
          pass


    except ValueError:


            yester_df= dictionary_yester[i]
            today_df = dictionary_now[i]


            yester_df.set_index('날짜',inplace=True)

            today_df.set_index('날짜',inplace=True)

            yester_df.update(today_df)


            tmpt = today_df.iloc[-1,:]

            yester_df = yester_df.append(tmpt)

            yester_df.reset_index(inplace=True)
            today_df.reset_index(inplace=True)

            #dictionary_yester[i] = yester_df

        ## 구글 API로 데이터 올리기
            # 2주간의 데이터만 구글 스프레드 시트에 올리겠다.

          #  for k in range(3):
           #     sleep(1)
            #    gc2.append_row(yester_df.iloc[-1-k,:].values.tolist()+[i])




            dictionary_yester[i] = yester_df


    except :
        print('API 에러',i)
        error.append(i)




    sleep(3)


  0%|          | 0/1 [00:00<?, ?it/s]

Http status: 403
failed to request
 타입 or 인덱스 에러,Value 에러: 이대맛집
Http status: 403
failed to request
Http status: 403
failed to request


In [37]:
# dictionary_now['lis에서 정의했던 키워드 입력']
dictionary_now['이대맛집']

KeyError: '이대맛집'

In [38]:
dictionary_yester['이대맛집']

KeyError: '이대맛집'